# Объяснение кросс-таблицы в Pandas

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/быстрое%20введение%20в%20pandas/Объяснение%20кросс-таблицы%20в%20Pandas.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

# Введение

Pandas предлагает несколько вариантов группировки и обобщения данных, но такое разнообразие вариантов может быть как благословением, так и проклятием. Все эти подходы являются мощными инструментами анализа данных, но не всегда понятно, использовать ли [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html), [`pivot_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) или [`crosstab`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) для построения сводной таблицы. 

Поскольку я [ранее рассматривал `pivot_tables`](https://dfedorov.spb.ru/pandas/%D0%A1%D0%B2%D0%BE%D0%B4%D0%BD%D0%B0%D1%8F%20%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0%20%D0%B2%20pandas.html), в этой статье будет обсуждаться функция `crosstab`, объяснено ее использование и показано, как ее можно использовать для быстрого суммирования данных.

> оригинал статьи Криса по [ссылке](https://pbpython.com/pandas-crosstab.html)

## Обзор 

Функция [`crosstab`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) создает таблицу кросс-табуляции, которая может показать частоту, с которой появляются определенные группы данных. 

В качестве быстрого примера в следующей таблице показано количество двух- или четырехдверных автомобилей, произведенных различными автопроизводителями:

<img src="https://github.com/dm-fedorov/pandas_basic/blob/master/pic/cross_tab.jpg?raw=true" alt="cross_tab" width="250"/>

В таблице видно, что набор данных содержит `32` автомобиля `Toyota`, из которых `18` четырехдверные и `14` двухдверные. Это относительно простая для интерпретации таблица, которая иллюстрирует, почему данный подход может стать мощным способом обобщения больших наборов данных.

Pandas упрощает этот процесс и позволяет настраивать таблицы несколькими способами. В оставшейся части статьи я расскажу, как создавать и настраивать эти таблицы.

Давайте начнем с импорта всех необходимых модулей:

In [ ]:
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

Теперь прочитаем [набор данных об автомобилях](https://archive.ics.uci.edu/ml/datasets/automobile) из репозитория машинного обучения UCI и внесем для ясности некоторые изменения в наименование меток.

> Этот набор данных из автомобильного ежегодника Уорда 1985 года состоит из трех типов записей: (а) спецификация автомобиля с точки зрения различных характеристик, (б) присвоенный ему рейтинг страхового риска, (в) его нормализованные потери при использовании по сравнению с другими автомобилями. 

In [ ]:
# Определим заголовки:
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

In [ ]:
# Прочитаем CSV-файл и преобразуем "?" в NaN:
df_raw = pd.read_csv("https://github.com/dm-fedorov/pandas_basic/blob/master/%D0%B1%D1%8B%D1%81%D1%82%D1%80%D0%BE%D0%B5%20%D0%B2%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B2%20pandas/data/imports-85.data?raw=true",
                     header=None, 
                     names=headers, 
                     na_values="?")
df_raw.head()

In [ ]:
# Быстро взглянем на все значения в данных:
df_raw.describe()

In [ ]:
# Определим список моделей, которые хотим рассмотреть:
models = ["toyota","nissan","mazda", "honda", "mitsubishi", "subaru", "volkswagen", "volvo"]

In [ ]:
# Создадим копию данных только с 8 ведущими производителями:
df = df_raw[df_raw.make.isin(models)].copy()
df.head()

В этом примере я хотел сократить таблицу, поэтому включил только 8 моделей, перечисленных выше. 

В качестве первого примера давайте воспользуемся `crosstab`, чтобы посмотреть, сколько различных стилей кузова изготовили эти автопроизводители в 1985 году (год, который содержится в этом наборе данных):

In [ ]:
pd.crosstab(df.make, df.body_style)

Функция `crosstab` может работать с массивами `numpy`, т.е. с `series` или столбцами во фрейме данных. 

В этом примере я передаю `df.make` для индекса кросс-таблицы и `df.body_style` для столбцов кросс-таблицы. Pandas подсчитывает количество вхождений каждой комбинации. Например, в этом наборе данных `Volvo` производит 8 седанов и 3 универсала.

Прежде чем мы пойдем дальше, более опытные читатели могут задаться вопросом, почему мы используем именно `crosstab`. Я кратко коснусь этого, показав два альтернативных подхода.

Во-первых, мы можем использовать `groupby`, а затем `unstack`, чтобы получить те же результаты:

In [ ]:
df.groupby(['make', 'body_style'])['body_style'].count().unstack().fillna(0)

Вывод для этого примера очень похож на кросс-таблицу, но потребовалось несколько дополнительных шагов, чтобы его правильно отформатировать.

Также можно сделать что-то подобное с помощью `pivot_table`:

In [ ]:
df.pivot_table(index='make', 
               columns='body_style', 
               aggfunc={'body_style':len}, 
               fill_value=0)

Обязательно прочтите мою [статью о pivot_tables](https://dfedorov.spb.ru/pandas/%D0%A1%D0%B2%D0%BE%D0%B4%D0%BD%D0%B0%D1%8F%20%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0%20%D0%B2%20pandas.html), если хотите понять, как это работает.

По-прежнему остается вопрос, зачем вообще использовать функцию `crosstab`? 

Короткий ответ заключается в том, что он предоставляет несколько удобных функций для упрощения форматирования и обобщения данных.

Более длинный ответ: бывает сложно запомнить все шаги для самостоятельного выполнения. 

> По моему опыту, важно знать о вариантах и использовать тот, который наиболее естественным образом вытекает из анализа. 

У меня был опыт, когда я пытался написать решение на основе `pivot_table`, а затем быстро получил то, что хотел, используя `crosstab`. 

Самое замечательное в pandas то, что после того, как данные помещены во фрейм, все манипуляции представляют собой 1 строку кода, поэтому вы можете экспериментировать.

## Углубляемся в кросс-таблицу

Одна из распространенных потребностей в кросс-таблице - это включение промежуточных итогов. 

Мы можем добавить их с помощью ключевого слова `margins`:

In [ ]:
pd.crosstab(df.make, 
            df.num_doors, 
            margins=True, 
            margins_name="Total")

Ключевое слово `margins` указало pandas добавлять `Total` (итог) для каждой строки, а также итог внизу. 

Я также передал значение в `margins_name` при вызове функции, потому что хотел обозначить результаты `Total` вместо значения по умолчанию `All`.

Во всех этих примерах подсчитывались отдельные случаи комбинаций данных. 

`crosstab` позволяет указывать значения для агрегирования. Чтобы проиллюстрировать это, мы можем рассчитать среднюю снаряженную массу автомобилей по типу кузова и производителю:

In [ ]:
pd.crosstab(df.make, 
            df.body_style, 
            values=df.curb_weight, 
            aggfunc='mean').round(0)

Используя `aggfunc='mean'` и `values=df.curb_weight`, мы говорим pandas применить функцию `mean` к весу снаряжения для всех комбинаций данных. Под капотом pandas группирует все значения вместе по `make` и `body_style`, а затем вычисляет среднее значение. В тех областях, где нет машины с такими значениями, отображается `NaN`. В этом примере я также округляю результаты.

Мы видели, как подсчитывать значения и определять средние значения. Однако есть еще один распространенный случай суммирования данных, когда мы хотим понять, сколько процентов от общего числа составляет каждая комбинация. Это можно сделать с помощью параметра `normalize`:

In [ ]:
pd.crosstab(df.make, 
            df.body_style, 
            normalize=True)

Эта таблица показывает нам, что `2.3%` от общей численности населения составляют хардтопы `Toyota`, а `6.25%` - седаны `Volvo`.

Параметр `normalize` еще умнее, т.к. он позволяет выполнять сводку отдельно для столбцов или строк. 

Например, если мы хотим увидеть, как стили корпуса распределяются по маркам:

In [ ]:
pd.crosstab(df.make, 
            df.body_style, 
            normalize='columns')

Взглянув только на колонку кабриолетов, можно увидеть, что `50%` автомобилей с откидным верхом производится `Toyota`, а остальные `50%` - `Volkswagen`. 

Мы можем сделать то же самое по строкам:

In [ ]:
pd.crosstab(df.make, 
            df.body_style, 
            normalize='index')

Это представление данных показывает, что из автомобилей `Mitsubishi` в этом наборе данных `69.23%` - это хэтчбеки, а оставшаяся часть (`30.77%`) - седаны.

Я надеюсь, вы согласитесь с тем, что эти приемы могут быть полезны во многих видах анализа.

## Группировка

Одна из наиболее полезных особенностей кросс-таблицы заключается в том, что вы можете передавать несколько столбцов фрейма данных, а pandas выполняет всю группировку за вас. 

Например, если мы хотим увидеть, как данные распределяются по переднему приводу (`fwd`) и заднему приводу (`rwd`), мы можем включить столбец `drive_wheels`, включив его в список допустимых столбцов во втором аргументе `crosstab`:

In [ ]:
pd.crosstab(df.make, 
            [df.body_style, df.drive_wheels])

То же самое можно сделать и с индексом:

In [ ]:
pd.crosstab([df.make, df.num_doors], 
            [df.body_style, df.drive_wheels],
            rownames=['Auto Manufacturer', "Doors"],
            colnames=['Body Style', "Drive Type"],
            dropna=False)

Я ввел пару дополнительных параметров для управления способом отображения вывода.

Во-первых, я задал определенные `rownames` и `colnames`, которые хочу включить в вывод. Это чисто для целей отображения, но может быть полезно, если имена столбцов во фрейме данных не конкретны.

Затем я использовал `dropna=False` в конце вызова функции. Причина, по которой я это включил, состоит в том, что я хотел убедиться, что включены все строки и столбцы, даже если в них все нули. Если бы я не включил его, то последний `Volvo`, двухдверный ряд, был бы исключен из таблицы.

Я хочу сделать последнее замечание по поводу этой таблицы. Она содержит много информации и может быть слишком трудной для интерпретации. Вот тут-то и приходит на помощь искусство науки о данных (или любого анализа), и вам нужно определить лучший способ представления данных. 

Приведу еще несколько примеров с различными параметрами:

In [ ]:
# Вы также можете использовать функции агрегирования при группировке:
pd.crosstab(df.make, 
            [df.body_style, df.drive_wheels], 
            values=df.curb_weight, 
            aggfunc='mean').fillna('-')

In [ ]:
# Вы можете использовать промежуточные итоги (margins) при группировке:
pd.crosstab(df.make, 
            [df.body_style, df.drive_wheels], 
            values=df.curb_weight, 
            aggfunc='mean', 
            margins=True,
            margins_name='Average').fillna('-').round(0)

Перейдем к заключительной части статьи.

## Визуализация

В последнем примере я соберу все воедино, показав, как выходные данные кросс-таблицы могут быть переданы на тепловую карту `Seaborn`, чтобы визуально обобщить данные.

В одной из наших кросс-таблиц мы получили 240 значений. Это слишком много, чтобы быстро анализировать, но если мы используем тепловую карту, то сможем легко интерпретировать данные. 

К счастью, `Seaborn` позволяет взять результат кросс-таблицы и визуализировать его:

In [ ]:
sns.heatmap(pd.crosstab([df.make, df.num_doors], 
                        [df.body_style, df.drive_wheels]),
            cmap="YlGnBu", 
            annot=True, 
            cbar=False);

Одним из действительно полезных аспектов этого подхода является то, что `Seaborn` сворачивает сгруппированные имена столбцов и строк, чтобы их было легче читать.

## Шпаргалка

Чтобы собрать все воедино, вот памятка, показывающая, как использовать все компоненты функции `crosstab`. 

Вы можете скачать PDF-версию по [ссылке](https://dfedorov.spb.ru/pandas/cheatsheet/crosstab_cheatsheet.pdf).

![](https://github.com/dm-fedorov/pandas_basic/blob/master/pic/crosstab_cheatsheet.png?raw=true)

# Заключение

Функция `crosstab` - полезный инструмент для обобщения данных. Функциональность пересекается с некоторыми другими инструментами pandas, но занимает полезное место в вашем наборе инструментов для анализа данных. Прочитав эту статью, вы сможете использовать ее в своем собственном анализе данных.